In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob
import gspread
import h3
from tqdm import tqdm
# from fcmeans import FCM
import matplotlib.pyplot as plt
from sklearn import manifold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import NMF
from sklearn.metrics import silhouette_score, silhouette_samples
import seaborn as sns
# import kmean
from sklearn.cluster import KMeans
from scipy.spatial import distance
import os

In [ ]:
# load tsne data
ROOTFOLDER = "/Users/yuan/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was"
DATA_FOLDER = f"{ROOTFOLDER}/_data/_curated/c_seg_hex"
DATA_EXPORT = f"{ROOTFOLDER}/_data/_curated/c_hex_cluster"
if not os.path.exists(DATA_EXPORT):
    os.makedirs(DATA_EXPORT)
BOUND_FOLDER = f"{ROOTFOLDER}/_data/_raw/r_boundary_osm"

# GRAPHIC_PATH = "/group/geog_pyloo/08_GSV/_graphic/cluster/allcities"
GRAPHIC_PATH = f"{ROOTFOLDER}/_graphic/cluster_within_city"
if not os.path.exists(GRAPHIC_PATH):
    os.makedirs(GRAPHIC_PATH)
res = 9

FILENAME = "c_seg_cat=31_res={res}_tsne.parquet"
df = pd.read_parquet(os.path.join(DATA_FOLDER, FILENAME.format(res = res)))
FILENAME_ORI = "c_seg_cat=31_res={res}.parquet"
df_ori = pd.read_parquet(os.path.join(DATA_FOLDER, FILENAME_ORI.format(res = res)))

In [ ]:
def plot_wss_kmean(data, N, comment):
    """Try Kmean instead"""
    wss = []
    distorsions = []
    for i in range(2,N):
        print("cluster: ", i)
        # fcm_vol = FCM(n_clusters=i, random_state=0)
        km = KMeans(n_clusters=i, random_state=0)
        km.fit(data)
        wss.append(km.inertia_)
        distorsions.append(sum(np.min(distance.cdist(data,
        km.cluster_centers_, 'euclidean'),axis=1)) /
        data.shape[0])
        
    sns.set(style="whitegrid")
    sns.set_context("paper", rc={"font.size":10,"axes.titlesize":12,"axes.labelsize":12})

    plt.figure(figsize=(6,4))
    plt.plot(range(2,N), wss, 'bx-', color='black')
    # plt.grid(True,alpha=.3)
    # plt.legend(framealpha=.3)
    plt.xlabel('Number of clusters $c$')
    plt.ylabel('Inertia')
    plt.title('The Elbow Method showing the optimal $c$')
    plt.savefig(os.path.join(GRAPHIC_PATH,f"elbow_volume_{N}_{comment}.png"), dpi=200, bbox_inches = 'tight')
    plt.show()
    return wss, distorsions

In [ ]:
data = df[['tsne_1', 'tsne_2']].copy()
# wss, distorsions = plot_wss_kmean(data, 30, "tsne-2d")

In [ ]:
# plt.figure(figsize=(6,4))
# plt.plot(range(2,30), distorsions, 'bx-', color='black')
# # plt.grid(True,alpha=.3)
# # plt.legend(framealpha=.3)
# plt.xlabel('Number of clusters $k$')
# plt.ylabel('Inertia')
# plt.title('The Elbow Method showing the optimal $k$')
# plt.savefig(os.path.join(GRAPHIC_PATH,f"elbow_volume_30_tsn-distorsion.png"), dpi=200, bbox_inches = 'tight')
# plt.show()

In [ ]:
N = 8
km = KMeans(n_clusters=N, random_state=0)
km.fit(data)
df[f'cluster_{N}'] = km.labels_
df_ori[f'cluster_{N}'] = km.labels_

In [ ]:
df.to_csv(os.path.join(DATA_FOLDER, "c_seg_cat=31_res={res}_tsne_cluster={n}.csv".format(res = res, n = N)), index=False)

In [ ]:
# N = 13
N = 8
df = pd.read_csv(os.path.join(DATA_FOLDER, "c_seg_cat=31_res={res}_tsne_cluster={n}.csv".format(res = res, n = N)))
df_ori[f'cluster_{N}'] = df[f'cluster_{N}']

In [ ]:
variables_sel_top1 = [
    'bike', 
             'building', 
             'bus', 
             'car',
       'grass', 
       'house', 
       'installation', 
       'lake+waterboday', 
       'light',
       'mountain+hill', 
       'person', 
       'pole', 
       'railing', 
       'road',
       'shrub', 
       'sidewalk', 
       'signage', 
       'sky',  
       'sportsfield',
       'table+chair', 
       'tower', 
       'traffic light', 
       'trashcan', 
       'tree', 
       'truck',
       'van', 
       'wall',
       'skyscraper',
       'window'
       ]

variable_mapping_2 = dict(zip(
       ['bike', 
             'building', 
             'bus', 
             'car',
       'grass', 
       'house', 
       'installation', 
       'lake+waterboday', 
       'light',
       'mountain+hill', 
       'person', 
       'pole', 
       'railing', 
       'road',
       'shrub', 
       'sidewalk', 
       'signage', 
       'sky',  
       'sportsfield',
       'table+chair', 
       'tower', 
       'traffic light', 
       'trashcan', 
       'tree', 
       'truck',
       'van', 
       'wall',
       'ground',
       'other',
       'skyscraper',
       'window'],
       ['bike', 
             'building', 
             'bus', 
             'car',
       'grass', 
       'house', 
       'furniture', 
       'nature', 
       'furniture', 
       'nature',  
       'person', 
       'furniture', 
       'railing', 
       'road',
       'shrub', 
       'sidewalk', 
       'signage', 
       'sky',  
       'sportsfield',
       'furniture', 
       'tower', 
       'furniture', 
       'furniture', 
       'tree', 
       'truck',
       'van', 
       'wall',
       'other',
       'other',
       'skyscraper',
       'building',])
)

In [ ]:
variables_sel_top2 = [
    'bike',
 'bus',
 'car',
 'house',
 'person',
 'railing',
 'road',
 'sidewalk',
 'truck',
 'van']

In [ ]:
variables_sel_top1_order = [
    'skyscraper',
     "bus",
 "road",'light','sidewalk', 
     
   
 "car", "van", 
        
       
    
     # nature and open space
     
      "building",
      "railing",
       
    'trashcan',
    'grass',
    'shrub', 
    'tree',
    
     'mountain+hill',
    # small scale building
        'house',
    
    # rural 
    
   
        
        
        # slow speed activities
         'bike',
         
         'installation', 
         'truck',
          'pole',
          'wall',
          
         
         
        'traffic light',
           
    'signage',
    'table+chair',
    'person',
    
    'window',
    'sportsfield',
    
    'sky',
    'lake+waterboday',
    
     
    
]

In [ ]:
# summarize the cluster type
df_summary = df_ori[variables_sel_top1_order+[f"cluster_{N}"]].groupby(f"cluster_{N}").mean().stack().reset_index()\
.rename(columns = {"level_1":"variable", 0:"std_value"})


df_summary_update = []
for v in df_summary["category"].unique():
    temp = df_summary[df_summary["category"]==v].reset_index(drop = True)
    temp["norm_value"] = (temp["std_value"] - temp["std_value"].min())/(temp["std_value"].max() - temp["std_value"].min())
    df_summary_update.append(temp)
    
df_summary_update = pd.concat(df_summary_update).reset_index()

df_heat = df_summary_update.pivot(
    columns= "category", values = "norm_value", index = f"cluster_{N}"
)[variables_sel_top1_order].sort_values('skyscraper', ascending = False)
df_heat



In [ ]:
# df_summary_update['category_mean'] = df_summary_update.groupby('category')['norm_value'].transform('median') 
# df_summary_update.sort_values('category_mean').drop_duplicates(subset = ['category'], keep = 'first')
# category_common_order = df_summary_update.sort_values('category_mean', ascending = False).drop_duplicates(subset = ['category'], keep = 'first')['category'].tolist()
# category_nature_to_build_order = {
#     "sky":1,
#     "lake+waterboday":1,
#     "mountain+hill":1,
#     "tree":2,
#     "grass":2,
#     "shrub":2,
#     "sportsfield":3,
    
#     "truck":4,
#     "van":4,
#     "car":4,
#     "bus":4,
#     "bike":5,
#     "person":5,
#     "trashcan":6,
#     "signage":6,
#     'traffic light':6,
#     "pole":6,
#     "light":6,
#     "table+chair":6,
#     "installation":6,
#     "wall":6,
#     "sidewalk":6,
    
#     "road":6,
#     "railing":6,
#     "building":6,
#     "house":6,
#     "window":6,
#     "skyscraper":6
# }

# df_summary_update['nature_order'] = df_summary_update['category'].map(category_nature_to_build_order)
# df_summary_update['common_order'] = df_summary_update['category'].map(dict(zip(category_common_order, range(len(category_common_order)))))
# df_summary_update

In [ ]:
# category_order_mix = df_summary_update.drop_duplicates(['category']).sort_values(['nature_order','common_order',], ascending = True)['category'].tolist()
# category_order_mix

In [ ]:
df_heat = df_summary_update.pivot(
    columns= "category", values = "norm_value", index = f"cluster_{N}"
)[variables_sel_top1_order].sort_values(variables_sel_top1_order[:5], ascending = False)
df_heat

In [ ]:
# use heatmap to visualize the cluster
fig, ax = plt.subplots(figsize = (22,4.5))
# use a gradient color
cluster_order = [0,5,3,6,2,1,7,4]
sns.heatmap(df_heat.loc[cluster_order], 
            cmap = "coolwarm", annot = True, fmt=".2f", 
            ax = ax)
# show axis annotation larger
ax.tick_params(axis='both', which='major', labelsize=12)
# show ticks to the left
ax.yaxis.tick_left()
# show ticks to the bottom
ax.xaxis.tick_top()
# rotate the x axis
plt.xticks(rotation=45)

In [ ]:
# cluster_order = [0,5,6,3,4,2,1,7,]
cluster_name = {
    0:"multi-modal dense urban center",
    5: "walkable dense urban center",
    3: "car-oriented urban center",
    6: "major connector infrastructure",
    2: "open space + built up",
    1: "park adjacent community",
    7:"human-scale park and open space",
    4: "bridge and tunnel"
} # need to find some example images to check the results

# Save geojson

In [ ]:
# use the husl color to map to each cluster
import seaborn as sns
color_palette = sns.color_palette("husl", N)
color_dict = dict(zip(range(N), color_palette))
color_palette

In [ ]:
# visualize one city
# get the geometry
from shapely.geometry import Polygon
import geopandas as gpd

def cell_to_shapely(cell):
    coords = h3.h3_to_geo_boundary(cell)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)


# loop through all cities and save the graphic and data
def get_result(city, N = N):
    cityabbr = city.lower().replace(" ", "")
    sample = df[df["city_lower"]==cityabbr].reset_index(drop = True)
    h3_geoms = sample['hex_id'].apply(lambda x: cell_to_shapely(x))
    df_sel_gdf = gpd.GeoDataFrame(sample[['hex_id',f"cluster_{N}"]], geometry = h3_geoms)
    df_sel_gdf.crs = "EPSG:4326"
    df_sel_gdf[f"cluster_{N}"] = df_sel_gdf[f"cluster_{N}"].astype(str)
    # use the color palette to map the cluster consistently
    fig, ax = plt.subplots(figsize = (8,8))
    for c in range(N):
        df_sel_gdf[df_sel_gdf[f"cluster_{N}"]==str(c)].plot(ax = ax, 
                                                            color = color_dict[c], 
                                                            linewidth = 0.5,
                                                            label = f"cluster_{c}",
                                                            legend = True)

    plt.title(city)
    # add the legend
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    exportfolder = os.path.join(GRAPHIC_PATH, "cluster="+str(N))
    if not os.path.exists(exportfolder):
        os.makedirs(exportfolder)
    plt.savefig(os.path.join(exportfolder, f"{city}_cluster={N}-tsn-res=9.png"), dpi=200, bbox_inches = 'tight')
    
    # df_sel_gdf.to_file(os.path.join(exportfolder,f"{city}_cluster={N}-tsn-res=9.geojson"), driver = "GeoJSON")
    return df_sel_gdf

# loop through all cities and save the graphic and data
def get_result_within(city, N = N):
    cityabbr = city.lower().replace(" ", "")
    sample = df[df["city_lower"]==cityabbr].reset_index(drop = True)
    h3_geoms = sample['hex_id'].apply(lambda x: cell_to_shapely(x))
    df_sel_gdf = gpd.GeoDataFrame(sample[['hex_id',f"cluster_{N}"]], geometry = h3_geoms)
    df_sel_gdf.crs = "EPSG:4326"
    df_sel_gdf[f"cluster_{N}"] = df_sel_gdf[f"cluster_{N}"].astype(str)
    # select the geometry within the boundary
    if "," in cityabbr:
        city_short = cityabbr.split(",")[0]
    else:
        city_short = cityabbr
    bound = gpd.read_file(os.path.join(BOUND_FOLDER, f"{city_short}.geojson"))
    assert bound.crs == df_sel_gdf.crs
    df_sel_gdf_within = gpd.sjoin(df_sel_gdf, bound[['geometry']])
    # use the color palette to map the cluster consistently
    fig, ax = plt.subplots(figsize = (8,8))
    for c in range(N):
        to_plot = df_sel_gdf_within[df_sel_gdf_within[f"cluster_{N}"]==str(c)]
        if to_plot.shape[0]>0:
            to_plot.plot(ax = ax, 
                                color = color_dict[c], 
                                linewidth = 0.5,
                                label = f"cluster_{c}",
                                legend = True)
        else:
            continue

    plt.title(city)
    # hide the axis
    plt.axis('off')
    # add the legend
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    exportfolder = os.path.join(GRAPHIC_PATH, "cluster="+str(N))
    data_export_folder = os.path.join(DATA_EXPORT, "cluster="+str(N))
    if not os.path.exists(data_export_folder):
        os.makedirs(data_export_folder)
    if not os.path.exists(exportfolder):
        os.makedirs(exportfolder)
    plt.savefig(os.path.join(exportfolder, f"{city}_cluster={N}-tsn-res=9.png"), 
                dpi=200, bbox_inches = 'tight')
    df_sel_gdf.to_file(os.path.join(data_export_folder,f"{city}_cluster={N}-tsn-res=9.geojson"), driver = "GeoJSON")
    return df_sel_gdf_within

# loop through all cities and save the graphic and data
def get_result_within_noplot(city, N = N):
    cityabbr = city.lower().replace(" ", "")
    sample = df[df["city_lower"]==cityabbr].reset_index(drop = True)
    h3_geoms = sample['hex_id'].apply(lambda x: cell_to_shapely(x))
    df_sel_gdf = gpd.GeoDataFrame(sample[['hex_id',f"cluster_{N}"]], geometry = h3_geoms)
    df_sel_gdf.crs = "EPSG:4326"
    df_sel_gdf[f"cluster_{N}"] = df_sel_gdf[f"cluster_{N}"].astype(str)
    # select the geometry within the boundary
    if "," in cityabbr:
        city_short = cityabbr.split(",")[0]
    else:
        city_short = cityabbr
    bound = gpd.read_file(os.path.join(BOUND_FOLDER, f"{city_short}.geojson"))
    assert bound.crs == df_sel_gdf.crs
    df_sel_gdf_within = gpd.sjoin(df_sel_gdf, bound[['geometry']])
    return df_sel_gdf_within

In [ ]:
fullcity = df["city_lower"].unique().tolist()
fullcity

In [ ]:
# allgdf = []
# for city in df["city_lower"].unique():
#     try:
#         df_sel_gdf = get_result(city)
#         allgdf.append(df_sel_gdf)
#     except:
#         print(f"Error in {city}")
# # allgdf = pd.concat(allgdf).reset_index(drop = True)

In [ ]:
allgdf = []
for city in fullcity:
    print(city)
    df_sel_gdf = get_result_within(city)
    df_sel_gdf['city_lower'] = city.split(",")[0]
    allgdf.append(df_sel_gdf)
allgdf = pd.concat(allgdf)\
    .reset_index(drop = True)

In [ ]:
allgdf.drop("geometry", axis = 1).to_csv(
    os.path.join(DATA_EXPORT, f"allcity_cluster={N}.csv"), index = False
)

In [ ]:
allgdf = []
for city in fullcity[:]:
    print(city)
    df_sel_gdf = get_result_within_noplot(city)
    df_sel_gdf['city_lower'] = city.split(",")[0]
    allgdf.append(df_sel_gdf)
allgdf = pd.concat(allgdf).reset_index(drop = True)
allgdf.head()

In [ ]:
allgdf.drop(columns = 'index_right', axis = 1, inplace = True)

In [ ]:
# allgdf.to_file(os.path.join(DATA_FOLDER, f"all_city_res=9_cluster={N}.geojson"), driver = "GeoJSON")
allgdf.drop('geometry', axis = 1).to_parquet(os.path.join(DATA_FOLDER, f"all_city_within_boundary_res=9_cluster={N}.parquet"))
allgdf.drop('geometry', axis = 1).to_csv(os.path.join(DATA_FOLDER, f"all_city_within_boundary_res=9_cluster={N}.csv"), index = False)

In [ ]:
df_sel_gdf = get_result_within("Hong Kong", N = N)

In [ ]:
df_sel_gdf = get_result_within("Singapore", N = N)
# df_sel_gdf.plot(figsize = (10,10), column = f"cluster_{N}", legend = True, linewidth = 0.1)

In [ ]:
df_sel_gdf = get_result_within("Paris", N = N)

In [ ]:
df_sel_gdf = get_result_within("London", N = N)

In [ ]:
get_result_within("Nairobi", N = N)

In [ ]:
get_result_within("hongkong", N = N)

In [ ]:
# check why singapore is so generic
df_ori[df_ori['city_lower']=='singapore'][['building', 'bus','person','sky', 'tree','skyscraper','road','car']].describe()

In [ ]:
df_ori[df_ori['city_lower']=='hongkong'][['building', 'bus','person','sky', 'tree','skyscraper','road','car']].describe()

In [ ]:
df[df['city_lower']=='london'].describe()

In [ ]:
df[df['city_lower']=='singapore'].describe()

In [ ]:
df[df['city_lower']=='hongkong'].describe()

In [ ]:
df_sel_gdf = get_result("Johannesburg", N = N)

## 2. Summarize the results across cities

In [ ]:
df_summary= pd.read_csv(os.path.join(DATA_FOLDER, "allcity_cluster=7-tsn-res=9.csv")).groupby(["city_lower","cluster"])\
    .size().reset_index().rename(columns = {0:"count"}).pivot(
    columns = "cluster", values = "count", index = "city_lower").fillna(0)
df_summary['total'] = df_summary.sum(axis = 1)
df_summary_prop = df_summary.copy()
for i in range(N):
    df_summary_prop[i] = df_summary_prop[i]/df_summary_prop['total']
df_summary_prop.to_csv(os.path.join(DATA_FOLDER, "allcity_cluster=7-tsn-res=9_city-level_summary.csv"))
df_summary_prop.sort_values(6, ascending = False).head(40)

In [ ]:
df_summary_prop.to_csv(os.path.join(DATA_FOLDER, "allcity_cluster=7-tsn-res=9_city-level_summary.csv"))